In [ ]:
import os
import numpy as np
from scipy.optimize import linear_sum_assignment
from ._base_metric import _BaseMetric
import _timing


class HOTA(_BaseMetric):
    """Class which implements the HOTA metrics.
    See: https://link.springer.com/article/10.1007/s11263-020-01375-2
    """
    
    # 클래스의 생성자
    def __init__(self, config=None):
        super().__init__()
        self.plottable = True # plot 생성 가능
        self.array_labels = np.arange(0.05, 0.99, 0.05) # HOTA 생 후 Plot 할 때 x축의 범위(0.05 ~ 0.99까지 0.05 간격)
        self.integer_array_fields = ['HOTA_TP', 'HOTA_FN', 'HOTA_FP'] # True Positive, False Negative, False Positive
        self.float_array_fields = ['HOTA', 'DetA', 'AssA', 'DetRe', 'DetPr', 'AssRe', 'AssPr', 'LocA', 'OWTA']
        self.float_fields = ['HOTA(0)', 'LocA(0)', 'HOTALocA(0)'] # 임계값(0)일 때의 HOTA, LocA, HOTALocA - 모델의 전반적인 성능(모델의 강건성) 체크용 지표
        self.fields = self.float_array_fields + self.integer_array_fields + self.float_fields
        self.summary_fields = self.float_array_fields + self.float_fields

    # 단일 시퀀스에 대한 HOTA Metric 계산
    # input -> data(딕셔너리)
    # data의 키
    # 'num_tracker_dets': 추적기(tracker)의 총 감지 개수 (스칼라)
    # 'num_gt_dets': 실제 GT(ground truth)의 총 감지 개수 (스칼라)
    # 'gt_ids': 각 시간 단계에서의 GT ID 목록 (리스트)
    # 'tracker_ids': 각 시간 단계에서의 추적기 ID 목록 (리스트)
    # 'similarity_scores': 각 시간 단계에서의 감지 간 유사도 점수 (2D 넘파이 배열)
    
    # data 딕셔너리에는 num_gt_ids와 num_tracker_ids도 존재 이는 코드에서 사용되는 변수와 관련된 정보
    
    @_timing.time
    def eval_sequence(self, data):
        """Calculates the HOTA metrics for one sequence"""

        # Initialise results
        # result 초기화
        res = {}
        for field in self.float_array_fields + self.integer_array_fields:
            res[field] = np.zeros((len(self.array_labels)), dtype=np.float)
        for field in self.float_fields:
            res[field] = 0

        # Return result quickly if tracker or gt sequence is empty
        
        # tracker가 빈 경우
        # res 리턴
        if data['num_tracker_dets'] == 0:
            res['HOTA_FN'] = data['num_gt_dets'] * np.ones((len(self.array_labels)), dtype=np.float)
            res['LocA'] = np.ones((len(self.array_labels)), dtype=np.float)
            res['LocA(0)'] = 1.0
            return res
        
        # ground truth 시퀀스가 빈 경우
        # res 리턴
        if data['num_gt_dets'] == 0:
            res['HOTA_FP'] = data['num_tracker_dets'] * np.ones((len(self.array_labels)), dtype=np.float)
            res['LocA'] = np.ones((len(self.array_labels)), dtype=np.float)
            res['LocA(0)'] = 1.0
            return res

        # Variables counting global association
        # 글로벌 연관성을 계산하는 변수들
        
        # 가능한 매칭 수(매칭 가능성)
        # potential_matches_count -> 2D 배열 (data['num_gt_ids'], data['num_tracker_ids'])
        potential_matches_count = np.zeros((data['num_gt_ids'], data['num_tracker_ids']))
        
        # ground truth의 수(등장한 수)
        # gt_id_count -> 2D 배열 (data['num_gt_ids'], 1)
        gt_id_count = np.zeros((data['num_gt_ids'], 1))
        
        # tracker의 수(등장한 수)
        # tracker_id_count -> 2D 배열 (1, data['num_tracker_ids'])
        tracker_id_count = np.zeros((1, data['num_tracker_ids']))

        # First loop through each timestep and accumulate global track information.
        # 각각의 타임스탬프를 진행하며 글로벌 Track 정보를 누적
        
        # 각각의 타임스탬프에 대해 gt와 tracker에 대해 반복
        for t, (gt_ids_t, tracker_ids_t) in enumerate(zip(data['gt_ids'], data['tracker_ids'])):
            # Count the potential matches between ids in each timestep
            # 각 시간 단계에서 ID 간의 가능한 매치를 계산
            # These are normalised, weighted by the match similarity.
            # 이들은 일치 유사성에 의해 가중치가 적용된 정규화된 값
            similarity = data['similarity_scores'][t] # 유사도 점수
            
            sim_iou_denom = similarity.sum(0)[np.newaxis, :] + similarity.sum(1)[:, np.newaxis] - similarity # IoU의 분모
            sim_iou = np.zeros_like(similarity) # IoU 분자
            
            # gt와 Tracker 쌍의 IoU를 계산할 때 사용
            # np.finfo('float').eps -> 표현 가능한 가장 작은 양수
            sim_iou_mask = sim_iou_denom > 0 + np.finfo('float').eps 
            sim_iou[sim_iou_mask] = similarity[sim_iou_mask] / sim_iou_denom[sim_iou_mask]
            # gt_id와 tracker_id 간의 잠재적 매칭 수를 계산하고 업데이트
            potential_matches_count[gt_ids_t[:, np.newaxis], tracker_ids_t[np.newaxis, :]] += sim_iou

            # Calculate the total number of dets for each gt_id and tracker_id.
            # 각 gt_id와 tracker_id에 대해 총 개수를 계산
            gt_id_count[gt_ids_t] += 1
            tracker_id_count[0, tracker_ids_t] += 1

        # Calculate overall jaccard alignment score (before unique matching) between IDs
        # ID 간의 전체 Jaccard 정렬 점수를 계산(교집합 / 합집합)
        global_alignment_score = potential_matches_count / (gt_id_count + tracker_id_count - potential_matches_count)
        matches_counts = [np.zeros_like(potential_matches_count) for _ in self.array_labels]

        # Calculate scores for each timestep
        # 타임스탬프별 점수 계산
        for t, (gt_ids_t, tracker_ids_t) in enumerate(zip(data['gt_ids'], data['tracker_ids'])):
            # Deal with the case that there are no gt_det/tracker_det in a timestep.
            
            # gt_ids_t가 없는 경우
            if len(gt_ids_t) == 0:
                for a, alpha in enumerate(self.array_labels):
                    res['HOTA_FP'][a] += len(tracker_ids_t)  # 해당 시간 단계에 존재하는 detector의 개수를 FP로 처리
                continue
            
            # tracker_ids_t가 없는 경우
            if len(tracker_ids_t) == 0:
                for a, alpha in enumerate(self.array_labels): # 해당 시간 단계에 존재하는 gt_id의 개수를 빠뜨린 것으로 처리
                    res['HOTA_FN'][a] += len(gt_ids_t)
                continue

            # Get matching scores between pairs of dets for optimizing HOTA
            # HOTA 최적화를 위해 각 det 쌍 간의 매칭 점수
            similarity = data['similarity_scores'][t]
            score_mat = global_alignment_score[gt_ids_t[:, np.newaxis], tracker_ids_t[np.newaxis, :]] * similarity

            # Hungarian algorithm to find best matches
            match_rows, match_cols = linear_sum_assignment(-score_mat)

            # Calculate and accumulate basic statistics
            for a, alpha in enumerate(self.array_labels):
                # 유사도가 alpha 이상의 매칭 점수를 가진 det를 식별하는 마스크
                actually_matched_mask = similarity[match_rows, match_cols] >= alpha - np.finfo('float').eps
                alpha_match_rows = match_rows[actually_matched_mask] # 일정 점수 이상의 행
                alpha_match_cols = match_cols[actually_matched_mask] # 일정 점수 이상의 열
                num_matches = len(alpha_match_rows) # 일정 점수 이상의 det의 수
                res['HOTA_TP'][a] += num_matches # TP
                res['HOTA_FN'][a] += len(gt_ids_t) - num_matches # FN = gt - TP
                res['HOTA_FP'][a] += len(tracker_ids_t) - num_matches # FP = tracking - FP
                if num_matches > 0:
                    res['LocA'][a] += sum(similarity[alpha_match_rows, alpha_match_cols]) # LocA -> Localization Accuracy(위치 정확도)
                    matches_counts[a][gt_ids_t[alpha_match_rows], tracker_ids_t[alpha_match_cols]] += 1

        # Calculate association scores (AssA, AssRe, AssPr) for the alpha value.
        # alpha 값에 대해 연관성 점수(AssA, AssRe, AssPr)를 계산
        # First calculate scores per gt_id/tracker_id combo and then average over the number of detections.
        # 먼저 각 gt_id/tracker_id 조합별로 점수를 계산한 다음 detection의 수로 평균을 구하기
        
        for a, alpha in enumerate(self.array_labels):
            matches_count = matches_counts[a]
            ass_a = matches_count / np.maximum(1, gt_id_count + tracker_id_count - matches_count)
            # AssA (Association Accuracy) -> 연관성의 정확도(accuracy) - (올바른 예측 수) / (전체 샘플 수)
            res['AssA'][a] = np.sum(matches_count * ass_a) / np.maximum(1, res['HOTA_TP'][a])
            ass_re = matches_count / np.maximum(1, gt_id_count)
            # AssRe (Association Recall) -> 연관성의 재현율(Recall) - (True Positive) / (True Positive + False Negative)
            res['AssRe'][a] = np.sum(matches_count * ass_re) / np.maximum(1, res['HOTA_TP'][a])
            ass_pr = matches_count / np.maximum(1, tracker_id_count)
            # AssPr (Association Precision) -> 연관성의 정밀도(Rrecision) - (True Positive) / (True Positive + False Positive)
            res['AssPr'][a] = np.sum(matches_count * ass_pr) / np.maximum(1, res['HOTA_TP'][a])

        # Calculate final scores
        # maximum은 0으로 나누는 것을 방지 하기 위해서 1e-10과 비교
        res['LocA'] = np.maximum(1e-10, res['LocA']) / np.maximum(1e-10, res['HOTA_TP'])
        res = self._compute_final_fields(res)
        return res


    # 모든 시퀀스의 평가 결과를 결합 - 시퀀스는 정수형 필드와 부동소수점 필드 모두를 의미
    def combine_sequences(self, all_res):
        """Combines metrics across all sequences"""
        res = {}
        # 정수형 시퀀스 result
        for field in self.integer_array_fields:
            res[field] = self._combine_sum(all_res, field)
        
        # AssRe, AssPr, AssA에 대한 가중 평균을 계산
        for field in ['AssRe', 'AssPr', 'AssA']:
            res[field] = self._combine_weighted_av(all_res, field, res, weight_field='HOTA_TP') # 가중 평균
        
        # 모든 시퀀스의 'LocA' 값을 'HOTA_TP' 값과 가중합으로 계산
        loca_weighted_sum = sum([all_res[k]['LocA'] * all_res[k]['HOTA_TP'] for k in all_res.keys()])
        res['LocA'] = np.maximum(1e-10, loca_weighted_sum) / np.maximum(1e-10, res['HOTA_TP'])
        res = self._compute_final_fields(res)
        return res
    
    
    # 모든 클래스에 대한 평가 결과를 평균내어 결합
    def combine_classes_class_averaged(self, all_res, ignore_empty_classes=False):
        """Combines metrics across all classes by averaging over the class values.
        If 'ignore_empty_classes' is True, then it only sums over classes with at least one gt or predicted detection.
        """
        res = {}
        # 정수 field
        for field in self.integer_array_fields:
            # 빈 class 제외하고 연산
            if ignore_empty_classes:
                res[field] = self._combine_sum(
                    {k: v for k, v in all_res.items()
                     if (v['HOTA_TP'] + v['HOTA_FN'] + v['HOTA_FP'] > 0 + np.finfo('float').eps).any()}, field)
            # 빈 class도 포함해서 연산
            else:
                res[field] = self._combine_sum({k: v for k, v in all_res.items()}, field)

        # 실수 field
        for field in self.float_fields + self.float_array_fields:
            # 빈 class 제외하고 연산
            if ignore_empty_classes:
                res[field] = np.mean([v[field] for v in all_res.values() if
                                      (v['HOTA_TP'] + v['HOTA_FN'] + v['HOTA_FP'] > 0 + np.finfo('float').eps).any()],
                                     axis=0)
            # 빈 class도 포함해서 연산
            else:
                res[field] = np.mean([v[field] for v in all_res.values()], axis=0)
        return res
    
    # 모든 클래스에 대한 평균 값을 계산하여 결과를 결합
    def combine_classes_det_averaged(self, all_res):
        """Combines metrics across all classes by averaging over the detection values"""
        res = {}
        
        # 정수 field
        for field in self.integer_array_fields:
            res[field] = self._combine_sum(all_res, field)
        
        # 실수 field
        for field in ['AssRe', 'AssPr', 'AssA']:
            res[field] = self._combine_weighted_av(all_res, field, res, weight_field='HOTA_TP')
        
        loca_weighted_sum = sum([all_res[k]['LocA'] * all_res[k]['HOTA_TP'] for k in all_res.keys()])
        res['LocA'] = np.maximum(1e-10, loca_weighted_sum) / np.maximum(1e-10, res['HOTA_TP'])
        res = self._compute_final_fields(res)
        return res

    @staticmethod
    def _compute_final_fields(res):
        """Calculate sub-metric ('field') values which only depend on other sub-metric values.
        This function is used both for both per-sequence calculation, and in combining values across sequences.
        """
        # Detection Recall(감지 재현율) - 실제 객체 중 정확히 탐지된 객체의 비율
        res['DetRe'] = res['HOTA_TP'] / np.maximum(1, res['HOTA_TP'] + res['HOTA_FN'])
        # Detection Precision(감지 정밀도) - 탐지된 객체 중에서 실제 객체와 일치하는 객체의 비율
        res['DetPr'] = res['HOTA_TP'] / np.maximum(1, res['HOTA_TP'] + res['HOTA_FP'])
        # Detection Accuracy(감지 정확도) - 실제 객체와 일체하는 객체의 비율
        res['DetA'] = res['HOTA_TP'] / np.maximum(1, res['HOTA_TP'] + res['HOTA_FN'] + res['HOTA_FP'])
        # Higher Order Tracking Accuracy (고차 트래킹 정확도)
        res['HOTA'] = np.sqrt(res['DetA'] * res['AssA'])
        # Optimal Waypoint Tracking Accuracy (최적 경로 트래킹 정확도)
        res['OWTA'] = np.sqrt(res['DetRe'] * res['AssA'])

        res['HOTA(0)'] = res['HOTA'][0]
        res['LocA(0)'] = res['LocA'][0]
        res['HOTALocA(0)'] = res['HOTA(0)']*res['LocA(0)']
        return res

    # 시각화
    def plot_single_tracker_results(self, table_res, tracker, cls, output_folder):
        """Create plot of results"""

        # Only loaded when run to reduce minimum requirements
        from matplotlib import pyplot as plt

        res = table_res['COMBINED_SEQ']
        styles_to_plot = ['r', 'b', 'g', 'b--', 'b:', 'g--', 'g:', 'm']
        for name, style in zip(self.float_array_fields, styles_to_plot):
            plt.plot(self.array_labels, res[name], style)
        plt.xlabel('alpha')
        plt.ylabel('score')
        plt.title(tracker + ' - ' + cls)
        plt.axis([0, 1, 0, 1])
        legend = []
        for name in self.float_array_fields:
            legend += [name + ' (' + str(np.round(np.mean(res[name]), 2)) + ')']
        plt.legend(legend, loc='lower left')
        out_file = os.path.join(output_folder, cls + '_plot.pdf')
        os.makedirs(os.path.dirname(out_file), exist_ok=True)
        plt.savefig(out_file)
        plt.savefig(out_file.replace('.pdf', '.png'))
        plt.clf()

## eval_sequence

- Input - data(평가할 시퀀스의 데이터를 담고 있는 딕셔너리)

    - 'gt_ids': 각 타임스텝에서의 gt_id를 포함하는 리스트 또는 배열
    - 'tracker_ids': 각 타임스텝에서의 tracker_id를 포함하는 리스트 또는 배열
    - 'similarity': 각 타임스텝에서의 객체 간 유사도를 나타내는 2D 배열, 배열의 크기는 (num_gt_ids, num_tracker_ids)

- Output - res

    - 'HOTA_TP': True Positive(TP) 값을 포함하는 배열
    - 'HOTA_FN': False Negative(FN) 값을 포함하는 배열
    - 'HOTA_FP': False Positive(FP) 값을 포함하는 배열
    - 'HOTA': HOTA(Higher Order Tracking Accuracy) 값을 포함하는 배열
    - 'DetA': DetA(Detection Accuracy) 값을 포함하는 배열
    - 'AssA': AssA(Association Accuracy) 값을 포함하는 배열
    - 'DetRe': Detection Recall 값을 포함하는 배열
    - 'DetPr': Detection Precision 값을 포함하는 배열
    - 'AssRe': Association Recall 값을 포함하는 배열
    - 'AssPr': Association Precision 값을 포함하는 배열
    - 'LocA': Localization Accuracy 값을 포함하는 배열
    - 'OWTA': Optimal Way of Tracking Accuracy 값을 포함하는 배열
    - 'HOTA(0)': alpha=0에서의 HOTA 값을 포함하는 배열
    - 'LocA(0)': alpha=0에서의 Localization Accuracy 값을 포함하는 배열
    - 'HOTALocA(0)': alpha=0에서의 HOTA+Localization Accuracy 값을 포함하는 배열

## combine_sequences 

- Input - all_res(각 시퀀스의 평가 결과를 담고 있는 딕셔너리의 리스트)
    - 각각의 딕셔너리는 eval_sequence의 아웃풋과 동일한 구조

- Output - res(모든 시퀀스를 결합한 평가 결과를 담고 있는 딕셔너리)

    - 정수형 배열 필드(integer_array_fields): 각 필드에 대한 값들을 모든 시퀀스에 대해 합산한 결과를 담고 있는 배열
    - 'AssRe', 'AssPr', 'AssA': 각 필드에 대한 값들을 모든 시퀀스에 대해 가중 평균한 결과를 담고 있는 배열
    - 'LocA': 모든 시퀀스의 'LocA' 값을 'HOTA_TP' 값과 가중합으로 계산한 결과를 담고 있는 배열

## combine_classes_class_averaged 

- Input - all_res(각 클래스의 평가 결과를 담고 있는 딕셔너리의 리스트)
    - 각각의 딕셔너리는 eval_sequence의 아웃풋과 동일한 구조
- Input - ignore_empty_classes(빈 클래스의 무시 여부, default 값은 False)

- Ouput - res(모든 클래스를 평균으로 결합한 평가 결과를 담고 있는 딕셔너리)    

## combine_classes_det_averaged 

- Input - all_res(딕셔너리의 리스트로 구성 각 딕셔너리는 클래스 레이블을 키로 갖고, 해당 클래스에 대한 결과 메트릭을 값으로 가짐)

- Output - res(딕셔너리)
    - res: 각 필드에 대한 결과 Metric 값을 저장하는 딕셔너리
    - 정수 필드 (self.integer_array_fields): 각 필드는 정수형 배열로 구성되어 있으며, 클래스별로 해당 필드의 값이 합산되어 저장
    - 실수 필드 (['AssRe', 'AssPr', 'AssA']): 각 필드는 실수 값으로 구성되어 있으며, 클래스별로 해당 필드의 값들의 가중 평균이 계산되어 저장
    - 'LocA': 실수 값으로 구성되어 있으며, 모든 클래스에 대한 'LocA' 값들의 가중합을 'HOTA_TP' 값으로 나눈 결과가 저장

In [ ]:
# 데이터셋에서 필요한 정보를 추출하여 data 배열을 생성
data = {
    'num_tracker_dets': num_tracker_dets, # tracker가 감지한 개체의 수(스칼라)
    'num_gt_dets': num_gt_dets, # 실제 개체의 수(스칼라)
    'gt_ids': gt_ids, # 실제 개체의 ID 리스트 - 1차원 배열(크기는 개체의 수)
    'tracker_ids': tracker_ids, # tracker의 ID 리스트 - 1차원 배열(크기는 tracker가 감지한 개체의 수) EX) [101, 102, 103, 104, 105]
    'similarity_scores': similarity_scores # 객체와 tracker가 감지한 개체와의 유사도 점수 - 2차원 배열 EX) [[0.8, 0.5, 0.2], [0.6, 0.3, 0.7], [0.4, 0.9, 0.1]]
}

# HOTA 객체 생성
hota = HOTA()

# eval_sequence 메서드를 사용하여 HOTA 계산
result = hota.eval_sequence(data)

# MOTA 값을 가져옴
mota = result['MOTA']